In [1]:
import argparse
import glob
from pathlib import Path
import open3d
from visual_utils import open3d_vis_utils as V
OPEN3D_FLAG = True
import numpy as np
import torch
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.datasets import DatasetTemplate
from pcdet.models import build_network, load_data_to_gpu
from pcdet.utils import common_utils

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
lidar_file = '/mnt/nas/users/dhvanil21040/nuscenes/samples/LIDAR_TOP/n008-2018-08-01-15-16-36-0400__LIDAR_TOP__1533151605548192.pcd.bin'
points = np.fromfile(lidar_file, dtype=np.float32).reshape(-1, 5)
points = points[:, :4]
# np.save('/mnt/nas/users/shreyas21563/data.npy', points) #Shreyas Location storage for .npy files
np.save('/mnt/nas/users/dhvanil21040/nuscenes_npy_lidar/n008-2018-08-01-15-16-36-0400__LIDAR_TOP__1533151605548192.npy', points) #Dhvanil Location storage for .npy files

In [6]:
class DemoDataset(DatasetTemplate):
    def __init__(self, dataset_cfg, class_names, training=False, root_path=None, logger=None, ext='.bin'):
        """
        Args:
            root_path:
            dataset_cfg:
            class_names:
            training:
            logger:
        """
        super().__init__(
            dataset_cfg=dataset_cfg, class_names=class_names, training=training, root_path=root_path, logger=logger
        )
        self.root_path = root_path
        self.ext = ext
        data_file_list = glob.glob(str(root_path / f'*{self.ext}')) if self.root_path.is_dir() else [self.root_path]
        data_file_list.sort()
        self.sample_file_list = data_file_list

    def __len__(self):
        return len(self.sample_file_list)

    def __getitem__(self, index):
        if self.ext == '.bin':
            points = np.fromfile(self.sample_file_list[index], dtype=np.float32).reshape(-1, 4)
        elif self.ext == '.npy':
            points = np.load(self.sample_file_list[index])
        else:
            raise NotImplementedError

        input_dict = {
            'points': points,
            'frame_id': index,
        }

        data_dict = self.prepare_data(data_dict=input_dict)
        return data_dict

In [7]:
# data_path = '/mnt/nas/users/shreyas21563/data.npy'
data_path = '/mnt/nas/users/dhvanil21040/nuscenes_npy_lidar/n008-2018-08-01-15-16-36-0400__LIDAR_TOP__1533151605548192.npy'
cfg_file = 'cfgs/nuscenes_models/cbgs_voxel0075_voxelnext.yaml'
ckpt = '/home/dhvanil21040/OpenPCDet/voxelnext_nuscenes_kernel1.pth'

In [8]:
cfg_from_yaml_file(cfg_file, cfg)
demo_dataset = DemoDataset(
        dataset_cfg=cfg.DATA_CONFIG, class_names=cfg.CLASS_NAMES, training=False,
        root_path=Path(data_path), ext='.npy'
    )

In [9]:
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=demo_dataset)

/home/dhvanil21040/miniconda3/envs/openpcdet/lib/python3.7/site-packages/torch/cuda/__init__.py:143: UserWarning: 
NVIDIA GeForce RTX 4090 with CUDA capability sm_89 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the NVIDIA GeForce RTX 4090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [10]:
logger = common_utils.create_logger()
model.load_params_from_file(filename=ckpt, logger=logger, to_cpu=True)
model.cuda()
model.eval()
print("")

2024-10-23 16:30:17,251   INFO  ==> Loading parameters from checkpoint /home/dhvanil21040/OpenPCDet/voxelnext_nuscenes_kernel1.pth to CPU
2024-10-23 16:30:17,302   INFO  Not updated weight backbone_3d.conv_input.0.weight: torch.Size([16, 3, 3, 3, 4])
2024-10-23 16:30:17,303   INFO  ==> Done (loaded 541/542)


In [11]:
with torch.no_grad():
    for idx, data_dict in enumerate(demo_dataset):
        data_dict = demo_dataset.collate_batch([data_dict])
        load_data_to_gpu(data_dict)
        pred_dicts, _ = model.forward(data_dict)

/home/dhvanil21040/OpenPCDet/pcdet/models/model_utils/centernet_utils.py:261: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  topk_classes = (topk_ind // K).int()


In [12]:
points=data_dict['points'][:, 1:].cpu().numpy()
ref_boxes=pred_dicts[0]['pred_boxes'].cpu().numpy()
ref_scores=pred_dicts[0]['pred_scores'].cpu().numpy()
ref_labels=pred_dicts[0]['pred_labels'].cpu().numpy()

In [13]:
import pickle
predictions = {
    "point": points,
    "ref_boxes": ref_boxes,
    "ref_scores": ref_scores,
    "ref_labels": ref_labels
}

# with open("/mnt/nas/users/shreyas21563/predictions.pkl", 'wb') as f:
#         pickle.dump(predictions, f)

with open("/mnt/nas/users/dhvanil21040/predictions_n008-2018-08-01-15-16-36-0400__LIDAR_TOP__1533151605548192.pkl", 'wb') as f:
        pickle.dump(predictions, f)